### Situação:

Usando os dados das licitações do ano de 2021, no Rio Grande do Sul, tentar fazer uma análise dos principais itens licitados e das áreas que receberam maior investimento por meio de licitações

Link: http://dados.tce.rs.gov.br/dataset?groups=licitacoes&q=consolidado

### Imports

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import spacy 
import nltk

In [4]:
files = os.listdir('/Users/ionasantana/Documents/GitHub/Artificial-Intelligence/Projects/NLP/2021')

In [5]:
files

['memcomissao.csv',
 'evento_lic.csv',
 'documento_lic.csv',
 'licitante.csv',
 'licitacao.csv',
 'item.csv',
 'pessoas.csv',
 'dotacao_lic.csv',
 'comissao.csv',
 'membrocons.csv',
 'proposta.csv',
 'item_prop.csv',
 'lote_prop.csv',
 'lote.csv']

In [7]:
licitacao = pd.read_csv('/Users/ionasantana/Documents/GitHub/Artificial-Intelligence/Projects/NLP/2021/' + files[4])
item = pd.read_csv('/Users/ionasantana/Documents/GitHub/Artificial-Intelligence/Projects/NLP/2021/' + files[5])

/Users/ionasantana/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ionasantana/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (8,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Testes

In [6]:
# pessoas = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[0])
# membrocons = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[1])
# comissao = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[2])
# memcomissao = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[3])

# licitante = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[5])
# dotacao_lic = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[6])
# evento_lic = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[7])
# documento_lic = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[8])
# lote = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[9])

# proposta = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[11])
# lote_prop = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[12])
# item_prop = pd.read_csv('/content/drive/MyDrive/NLP/2021/' + files[13])

#Testes
# pessoas.head(3)
# # pessoas.shape # 727385
# # pessoas.isna().sum()
# membrocons.head(3)
# # membrocons.shape # 377
# # membrocons.isna().sum()
# comissao.head(3)
# # comissao.shape # 13243
# # comissao.isna().sum()
# memcomissao.head(3)
# # memcomissao.shape # 68528
# # memcomissao.isna().sum()
# licitante.head(3) # que ou quem faz o lance ou oferta de compra pelo preço que indica;
# # licitante.shape # 147454
# # licitante.isna().sum()
# licitante['TP_RESULTADO_HABILITACAO'].unique()
# dotacao_lic.head(3)
# # dotacao_lic.shape # 123371
# # dotacao_lic.isna().sum()
# # evento_lic.head(3)
# # evento_lic.shape # 252892
# # evento_lic.isna().sum()
# proposta.head(3)
# # proposta.shape # 147454
# # proposta.isna().sum()
# lote_prop.head(3)
# # lote_prop.shape # 308350
# # lote_prop.isna().sum()
# item_prop.head(3)
# # item_prop.shape # 2039001
# # item_prop.isna().sum()
# # aux = item_prop[item_prop['DT_HOMOLOGACAO'] == '0001-01-01'] #Hum?
# # aux['VL_TOTAL_ITEM'].sum() #286.736.736

### Pré-processamento Dados

In [8]:
cols = ['CD_ORGAO', 'NM_ORGAO', 'NR_LICITACAO', 'CD_TIPO_MODALIDADE','DS_OBJETO', 'VL_LICITACAO', 'BL_COVID19']
licitacao_21 = licitacao[cols]

In [9]:
licitacao_21 = licitacao_21.dropna()

In [10]:
licitacao_21.head(3)

,CD_ORGAO,NM_ORGAO,NR_LICITACAO,CD_TIPO_MODALIDADE,DS_OBJETO,VL_LICITACAO,BL_COVID19
0,53100,PM DE NOVA PALMA,1.0,TMP,CONTRATAÇÃO GLOBAL DE EMPRESA PARA CONSTRUÇÃO ...,259765.0,N
1,86800,PM DE BOA VISTA DO SUL,2.0,PRP,Contratação de empresa para prestação de servi...,390625.0,N
2,45003,SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL,46.0,PRP,Aquisição de 80.000 (oitenta mil) quilogramas ...,833800.0,N


In [11]:
item_licitacao = item.iloc[:, 0:12]
item_licitacao['BL_COVID19'] = item['BL_COVID19'] 

In [12]:
item_licitacao.isna().sum()

CD_ORGAO                    0
NR_LICITACAO                0
ANO_LICITACAO               0
CD_TIPO_MODALIDADE          0
NR_LOTE                     0
NR_ITEM                     0
NR_ITEM_ORIGINAL            0
DS_ITEM                     1
QT_ITENS                    0
SG_UNIDADE_MEDIDA           0
VL_UNITARIO_ESTIMADO    39446
VL_TOTAL_ESTIMADO       39446
BL_COVID19                  0
dtype: int64

In [13]:
item_licitacao = item_licitacao.dropna()

In [14]:
len(licitacao_21), len(item_licitacao)

(87469, 932800)

In [15]:
licitacao_21['covid'] = np.where((licitacao_21['BL_COVID19'] == 'S'), ' covid19', '')
licitacao_21['DS_OBJETO'] = licitacao_21['DS_OBJETO'] + licitacao_21['covid']

In [16]:
licitacao_21 = licitacao_21.drop(['BL_COVID19', 'covid'], axis = 1)

In [18]:
item_licitacao['covid'] = np.where((item_licitacao['BL_COVID19'] == 'S'), ' covid19', '')
item_licitacao['DS_ITEM'] = item_licitacao['DS_ITEM'] + item_licitacao['covid']

In [19]:
item_licitacao = item_licitacao.drop(['BL_COVID19', 'covid'], axis = 1)

In [20]:
item_licitacao['ANO_LICITACAO'].unique()
item_licitacao.drop(['ANO_LICITACAO'], axis = 1).head(3)

,CD_ORGAO,NR_LICITACAO,CD_TIPO_MODALIDADE,NR_LOTE,NR_ITEM,NR_ITEM_ORIGINAL,DS_ITEM,QT_ITENS,SG_UNIDADE_MEDIDA,VL_UNITARIO_ESTIMADO,VL_TOTAL_ESTIMADO
0,44500,274.0,CPC,1,1,1,Credenciamento de academias de ginástica e mus...,1,UN,50.00,50.00
1,56900,25.0,TMP,1,1,1.1,PLACA DE OBRA (PARA CONSTRUCAO CIVIL) EM CHAPA...,6,M2,272.49,1634.94
2,56900,25.0,TMP,1,2,1.2,TAPUME COM COMPENSADO DE MADEIRA. AF_05/2018,71,M2,128.69,9136.99


### Pré-processamento Texto

In [33]:
# Libs adicionais de NLP
import spacy 
# import pt_core_news_lg
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

In [39]:
nlp=spacy.load("pt_core_news_sm")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ionasantana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
stopwords = nltk.corpus.stopwords.words('portuguese')

filter_out = stopwords + ['kg', 'm', 'cm', 'mm', 'mg', 'ml', 'm²', '-', '+', '=', 'mcg', 'nº', 'g', 'ª', 'º']

<table ><tr><th>Character</th><th>Description</th><th>Example Pattern Code</th><th >Exammple Match</th></tr>

<tr ><td><span >\d</span></td><td>A digit</td><td>file_\d\d</td><td>file_25</td></tr>

<tr ><td><span >\w</span></td><td>Alphanumeric</td><td>\w-\w\w\w</td><td>A-b_1</td></tr>



<tr ><td><span >\s</span></td><td>White space</td><td>a\sb\sc</td><td>a b c</td></tr>



<tr ><td><span >\D</span></td><td>A non digit</td><td>\D\D\D</td><td>ABC</td></tr>

<tr ><td><span >\W</span></td><td>Non-alphanumeric</td><td>\W\W\W\W\W</td><td>*-+=)</td></tr>

<tr ><td><span >\S</span></td><td>Non-whitespace</td><td>\S\S\S\S</td><td>Yoyo</td></tr></table>

<table ><tr><th>Character</th><th>Description</th><th>Example Pattern Code</th><th >Exammple Match</th></tr>

<tr ><td><span >+</span></td><td>Occurs one or more times</td><td>	Version \w-\w+</td><td>Version A-b1_1</td></tr>

<tr ><td><span >{3}</span></td><td>Occurs exactly 3 times</td><td>\D{3}</td><td>abc</td></tr>



<tr ><td><span >{2,4}</span></td><td>Occurs 2 to 4 times</td><td>\d{2,4}</td><td>123</td></tr>



<tr ><td><span >{3,}</span></td><td>Occurs 3 or more</td><td>\w{3,}</td><td>anycharacters</td></tr>

<tr ><td><span >\*</span></td><td>Occurs zero or more times</td><td>A\*B\*C*</td><td>AAACC</td></tr>

<tr ><td><span >?</span></td><td>Once or none</td><td>plurals?</td><td>plural</td></tr></table>

In [69]:
def pre_processing(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^\w\s]", "", sentence)
    sentence = re.sub(r"[0-9]+", "", sentence)
    sentence = re.sub(r"/+", "", sentence)
    sentence = re.sub(r" {2,}", " ", sentence)

    #Tokenização
    doc = nlp(sentence)

    tokens=[token.text for token in doc]
    tokens=[token for token in tokens if token not in filter_out]

    return tokens


In [45]:
item_licitacao['DS_ITEM_TOKEN'] = item_licitacao['DS_ITEM'].apply(lambda sentece: pre_processing(sentece))


FileNotFoundError: [Errno 2] No such file or directory: 'base/item_licitacao_token'

In [47]:
item_licitacao.to_csv('item_licitacao_token.csv')

In [48]:
licitacao_21['DS_OBJETO_TOKEN'] = licitacao_21['DS_OBJETO'].apply(lambda sentece: pre_processing(sentece))


In [49]:
licitacao_21.to_csv('licitacao_21_token.csv')

### Construção dos modelos

In [51]:
licitacao_21.head(3) #retirar os (), depois?

,CD_ORGAO,NM_ORGAO,NR_LICITACAO,CD_TIPO_MODALIDADE,DS_OBJETO,VL_LICITACAO,DS_OBJETO_TOKEN
0,53100,PM DE NOVA PALMA,1.0,TMP,CONTRATAÇÃO GLOBAL DE EMPRESA PARA CONSTRUÇÃO ...,259765.0,[contrataçãoglobaldeempresaparaconstruçãodopar...
1,86800,PM DE BOA VISTA DO SUL,2.0,PRP,Contratação de empresa para prestação de servi...,390625.0,[contrataçãodeempresaparaprestaçãodeserviçosco...
2,45003,SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL,46.0,PRP,Aquisição de 80.000 (oitenta mil) quilogramas ...,833800.0,[aquisiçãodeoitentamilquilogramasdeortopolifos...


In [52]:
item_licitacao.head(3)

,CD_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_LOTE,NR_ITEM,NR_ITEM_ORIGINAL,DS_ITEM,QT_ITENS,SG_UNIDADE_MEDIDA,VL_UNITARIO_ESTIMADO,VL_TOTAL_ESTIMADO,DS_ITEM_TOKEN
0,44500,274.0,2021,CPC,1,1,1,Credenciamento de academias de ginástica e mus...,1,UN,50.00,50.00,[credenciamentodeacademiasdeginásticaemusculaç...
1,56900,25.0,2021,TMP,1,1,1.1,PLACA DE OBRA (PARA CONSTRUCAO CIVIL) EM CHAPA...,6,M2,272.49,1634.94,[placadeobraparaconstrucaocivilemchapagalvaniz...
2,56900,25.0,2021,TMP,1,2,1.2,TAPUME COM COMPENSADO DE MADEIRA. AF_05/2018,71,M2,128.69,9136.99,"[tapumecomcompensadodemadeiraaf, _]"


In [64]:
doc = item_licitacao['DS_ITEM_TOKEN'][0]
print(doc)
for token in doc:
    print(token)

['credenciamentodeacademiasdeginásticaemusculaçãoououtroestabelecimentoqueministremexercíciosfísicosnoâmbitodomunicípiodecanoasovalorderseráodesembolsadopelomunicípioporcadaservidormatriculadoemestabelecimentocredenciadonomunicípioeescolhidopeloservidor']
credenciamentodeacademiasdeginásticaemusculaçãoououtroestabelecimentoqueministremexercíciosfísicosnoâmbitodomunicípiodecanoasovalorderseráodesembolsadopelomunicípioporcadaservidormatriculadoemestabelecimentocredenciadonomunicípioeescolhidopeloservidor


### Análise dos dados

### Conclusão

Quais foram as áreas com maiores investimentos?

E quais foram os principais itens licitados nelas?